In [5]:
import pandas as pd
import numpy as np 
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import xgboost as xgb
from skopt.space import Real, Categorical, Integer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from skopt import BayesSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score,roc_auc_score

In [6]:
df1 = pd.read_csv('Data.csv')

In [7]:
df1['Address'] = df1['Address'].apply(lambda x : 'NA' if pd.isnull(x) else x)
df1['Return'] = df1['Visit.No'].apply(lambda x: True if x>1 else False)

In [8]:
cal_session = min(df1[df1['Session']==2].index)
df1['His.ServTime'] = pd.NA
for i in range(cal_session,len(df1)):#Compute Hist. ServTime, starting from Session 2
    session = df1.loc[i,'Session']
    if df1.loc[i,'Return']:# if it is a Return patient
        id = df1.loc[i,'ID']
        df_temp = df1[(df1['ID']==id)&(df1['Session']<session)]['ServTime'].values #Only use data from previous sessions
        if len(df_temp)!=0:
            df1.loc[i,'His.ServTime'] = np.mean(df_temp)
        else:
            df1.loc[i,'His.ServTime'] = pd.NA #Mark as NA. Will imputate it with 15min (one-slot) later
    else:#A new patient
        df_temp = df1[(df1['Return']==False)&(df1['Session']<session)]['ServTime'].values
        df1.loc[i,'His.ServTime'] = np.mean(df_temp)

In [9]:
df1['His.ServTime'] = df1['His.ServTime'].fillna(900)#Imputate it with 15min (one-slot)

C:\Users\JYW2018\AppData\Local\Temp\ipykernel_29284\2847841537.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['His.ServTime'] = df1['His.ServTime'].fillna(900)#Imputate it with 15min (one-slot)


In [10]:
train_end_idx = max(df1[df1['Session']==194].index)
session1_len = len(df1[df1['Session']==1])
def data_process(df,feature,y):
    data = df[feature]
    data_x = data.drop([y],axis = 1)
    data_y = data[y]
    x_train = data_x.loc[:train_end_idx,].copy()
    x_test = data_x.loc[train_end_idx+1:,].copy()
    y_train = data_y.loc[:train_end_idx,].copy()
    y_test = data_y.loc[train_end_idx+1:,].copy()
    for i in ['His.ServTime','Visit.No']:#standardization of continuous variables
        x1 = np.array(x_train.loc[:,i]).reshape(-1,1)
        scaler = StandardScaler()
        scaler.fit(x1)
        x_train[i+'_scaled'] = scaler.transform(x1).reshape(1,-1)[0]
        x2 = np.array(x_test.loc[:,i]).reshape(-1,1)
        x_test[i+'_scaled'] = scaler.transform(x2).reshape(1,-1)[0]
    x_train = x_train.drop(['Visit.No','His.ServTime'],axis = 1)
    x_test = x_test.drop(['Visit.No','His.ServTime'],axis = 1)
    x_train = pd.get_dummies(x_train,columns = ['Gender','Address'], drop_first=True)#one-hot encoding
    x_test = pd.get_dummies(x_test,columns = ['Gender','Address'], drop_first=True)
    return x_train, x_test, y_train, y_test

In [11]:
input_feature = ['Visit.No','M.Cancer','S.Cancer','Gender','Return','His.ServTime','Address','ServTime']
x_train, x_test, y_train, y_test = data_process(df1,input_feature,'ServTime')